In [25]:
import pandas as pd

path = 'MoviesDataSet/'

movies = pd.read_csv(path + 'movies_metadata.csv')

# Filter out any unreleased movies
movies = movies[movies['status'] != 'Released']

movies.head()

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
189,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",NaN,56088,tt0114435,en,The Show,A documentary about the culture of hip-hop. Th...,...,1995-08-25,0.0,93.0,[],NaN,What Happens On Stage Is Only Half The Story!,The Show,False,5.3,2.0
682,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,48260,tt0112604,en,Butterfly Kiss,Eunice is walking along the highways of northe...,...,1995-08-18,0.0,88.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Rumored,NaN,Butterfly Kiss,False,7.4,5.0
767,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}]",http://www.tsquare.tv/film/,52059,tt0113147,en,The Gate of Heavenly Peace,The Gate of Heavenly Peace is a feature-length...,...,1995-01-01,0.0,180.0,[],NaN,NaN,The Gate of Heavenly Peace,False,6.7,3.0
775,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,44495,tt0072362,en,A Very Natural Thing,"When David, an ex-monk still in his twenties m...",...,1974-01-01,0.0,80.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Rumored,NaN,A Very Natural Thing,False,6.3,3.0
1032,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,38191,tt0117781,en,The Sunchaser,A young half-Navajo convict (Jon Seda) dying o...,...,1996-05-29,0.0,122.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Rumored,It began as a kidnapping. It became a journey ...,The Sunchaser,False,6.1,8.0


In [26]:
# Filter down the columns to those we really care about (losing belongs_to_collection, homepage, original_language, poster_path, spoken_languages, original_title, video, status
# Note: budget and revenue are interesting, but are mostly 0's so are unusable here, so those are lost too
movies = movies[['id', 'imdb_id',  'title', 'adult', 'genres', 'popularity', 'overview', 'production_companies', 'production_countries', 'release_date', 'runtime', 'tagline', 'vote_average', 'vote_count']]

movies.head()

,id,imdb_id,title,adult,genres,popularity,overview,production_companies,production_countries,release_date,runtime,tagline,vote_average,vote_count
189,56088,tt0114435,The Show,False,"[{'id': 99, 'name': 'Documentary'}]",0.159596,A documentary about the culture of hip-hop. Th...,[],[],1995-08-25,93.0,What Happens On Stage Is Only Half The Story!,5.3,2.0
682,48260,tt0112604,Butterfly Kiss,False,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",0.877853,Eunice is walking along the highways of northe...,"[{'name': 'Dan Films', 'id': 1854}, {'name': '...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'}]",1995-08-18,88.0,NaN,7.4,5.0
767,52059,tt0113147,The Gate of Heavenly Peace,False,"[{'id': 99, 'name': 'Documentary'}]",0.10705,The Gate of Heavenly Peace is a feature-length...,[],[],1995-01-01,180.0,NaN,6.7,3.0
775,44495,tt0072362,A Very Natural Thing,False,"[{'id': 18, 'name': 'Drama'}]",0.147662,"When David, an ex-monk still in his twenties m...",[],"[{'iso_3166_1': 'US', 'name': 'United States o...",1974-01-01,80.0,NaN,6.3,3.0
1032,38191,tt0117781,The Sunchaser,False,"[{'id': 18, 'name': 'Drama'}]",2.35809,A young half-Navajo convict (Jon Seda) dying o...,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1996-05-29,122.0,It began as a kidnapping. It became a journey ...,6.1,8.0


In [27]:
movies.describe()

,runtime,vote_average,vote_count
count,440.000000,446.000000,446.000000
mean,79.525000,5.065695,9.627803
std,40.665022,2.566747,27.142579
min,0.000000,0.000000,0.000000
25%,75.750000,4.000000,2.000000
50%,90.000000,5.800000,4.000000
75%,101.000000,6.700000,9.000000
max,180.000000,10.000000,453.000000
